In [ ]:
import math
import codecs
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
from collections import Counter

def count_offset (k):
    
    ''' Рассчитывает необходимый оффсет для скролла ленты ВК.
    Оффсет начинается с 15 и дальше + 10, то есть 15, 25, 35 и т.д.'''
    
    import math
    needed_offset = k * math.ceil (k/5)

    if k%10 == 5:
        return k
    elif k%10 in range (0,5):
        return (round(k,-1)+5)
    elif k%10 in range (6,10):
        return (round(k,-1)+5)
    
# ОБЩИЙ КОД

def group_wall_post_texts ():
    
    ''' Функция принимает на вход id из URL паблика и число необходимых постов, а возвращает тексты этих постов. '''
    
    group_url = input ('Введите id паблика из URL (Например, из "https://m.vk.com/dandaidaodu" нужно ввести "dandaidaodu"): ')
    
    print (f'Собираем посты паблика https://m.vk.com/{group_url}')
    
    want_to_get = int(input('Введите, сколько постов Вы хотите собрать (положительное целое число): '))

    # final_url_with_offset =  f"https://m.vk.com/{group_url}?offset={count_offset(want_to_get)}&own=1"  
    #Выводит URL для одного оффсета

    list_of_offsets_needed = []

    indesa = int((count_offset(want_to_get)-5)/10) # Сколько оффсетов >5 нужно подсобрать 

    for i in range (0, indesa+1):
        list_of_offsets_needed.append (5+10*i) # Собираем оффсеты в единый список по типу [5, 15, 25, 35, 45]
        
    url_list_of_offsets_needed = [f"https://m.vk.com/{group_url}"] # Сюда собираются URL с оффсетами. По умолчанию указан URL без оффсета

    for i in list_of_offsets_needed:
        final_url_with_offset =  f"https://m.vk.com/{group_url}?offset={i}&own=1"
        url_list_of_offsets_needed.append (final_url_with_offset) # Сбор URL с оффсетами
        
    all_texts_with_offsets = [] 

    for i in url_list_of_offsets_needed: 

        response = requests.request("GET", i) 

        # Далее кусок кода, написанный при выполнении предыдущего задания

        document = BeautifulSoup(response.text)
        resultset = document.find_all('div', class_ = 'pi_text') 

        for j in list(resultset):
            all_texts_with_offsets.append (j.text) # Парсим тексты постов со всех оффсетов

    print (f'Итого собрано {len (all_texts_with_offsets)} постов')  


    needed_amound_of_texts = all_texts_with_offsets [:want_to_get] # Ограничиваем собранное до нужного числа постов
    
    print (f'Если собрано и выведено в итог меньшее число постов, чем Вы хотели собрать, то это значит, что либо в паблике были посты без текста, либо стена паблика закончилась, и это число ненайденных постов. Здесь эта разница составляет {want_to_get-len(needed_amound_of_texts)}')


    print (f'Выведено в итог {len(needed_amound_of_texts)} постов')

    return needed_amound_of_texts

def agg_group_member_ids (group_id, access_token, offset_original = 100, offset_wanted = 100):

    ''' Cобирает id подписчиков паблика в список. 
    Обязательные аргументы: 1) id группы — group_id — смотреть тут: https://vk.com/dev/groups.getById — int
    2) access_token — токен, полученный с помощью Standalone приложения — https://vk.com/dev/implicit_flow_user — str
    Необязательные аргументы: 1) offset_original — нужен на финальном этапе, чтобы не потерять последних подписчиков — int
    2) offset_wanted — шаг, на который итерируются-продвигаются действия — int
    '''

    import time 

    id_list= []
    #offset_original = 100
    #offset_wanted = 100
    offset = 0

    url = f"https://api.vk.com/method/groups.getMembers?group_id={group_id}&count=100&offset={offset}&access_token={access_token}&v=5.124"
    count_posts_original=requests.request("GET", url).json()['response']['count']

    while True:
        time.sleep(1)       #делаем между запросами перерыв в секунду
        url = f"https://api.vk.com/method/groups.getMembers?group_id={group_id}&count=100&offset={offset}&access_token={access_token}&v=5.124"
        # print (offset)
        url_formatted = url.format(offset = offset)
        # print (url_formatted)

        for post in requests.request("GET", url_formatted).json()['response']['items']:
            # print (post)
            id_list.append(post)
        if len (id_list) >= (count_posts_original-offset_original):
            offset_new=offset+offset_wanted
            url_formatted_final = f"https://api.vk.com/method/groups.getMembers?group_id={group_id}&count=100&offset={offset_new}&access_token={access_token}&v=5.124"
            for post in requests.request("GET", url_formatted_final).json()['response']['items']:
                id_list.append(post)
            break
        else:
            offset=offset+offset_wanted

    clear_list = list(set(id_list))

    print(f'Итого собрано id: {len(clear_list)}')
    
    return clear_list

def gender_counts (group_id, access_token):
    
    ''' Функция принимает на вход токен доступа и id паблика (group_id). 
    Возвращает подсчитанное количество мужчин и женщин. '''
    
    group = agg_group_member_ids (group_id, access_token)
    group_len = len(group)
    a = 0
    b = 300
    all_sexes = []
    while a < group_len:
        time.sleep(1)
        group_string = str(group[a:b])
        group_string = group_string.replace("[", "").replace("]", "").replace(" ", "")
        group_sex = f'https://api.vk.com/method/users.get?user_ids={group_string}&fields=sex&access_token={access_token}&v=5.124'
        response = requests.request("GET", group_sex).json()['response']
        for i in response:
            k = i['sex']
            all_sexes.append(k)
        a = a+300
        b = b+300
    men = Counter(all_sexes)[2]
    women = Counter(all_sexes) [1]
    print (f'Всего в паблике {men} мужчин')
    print (f'Всего в паблике {women} женщин')
    return response, men, women

def param_counts (group_id, access_token, param):
    
    '''
    3. param — параметр, который нужно собрать о пользователях. Только один!!!! Иначе будет перегруз оперативной памяти, 
    файл ляжет, а данные будет трудно достать. Допустимые параметры на выбор: sex, bdate, city, country, home_town. 
    Вводятся как строка'''
    
    group = agg_group_member_ids (group_id, access_token)
    group_len = len(group)
    a = 0
    b = 300
    all_params = []
    iterel = 1
    while a < group_len:
        print ('ВНИМАНИЕ, ИТЕРАЦИЯ №',iterel)
        time.sleep(1)
        group_string = str(group[a:b])
        group_string = group_string.replace("[", "").replace("]", "").replace(" ", "")
        group_param = f'https://api.vk.com/method/users.get?user_ids={group_string}&fields={param}&access_token={access_token}&v=5.124'
        response = requests.request("GET", group_param).json()['response']
        # print (response)
        for i in response:
            try:
                k = i[param]
                
                if param == 'bdate':
                    if len (k) > 8:
                        all_params.append (k[-4:]) 
                else:                
                    all_params.append(k)
                # print (all_params)
            except KeyError:
                # print ('Oh No')
                pass
        a = a+300
        b = b+300
        iterel = iterel + 1
    # print (all_params)
    if param in ['sex', 'bdate','home_town']:
        counter_result = Counter(all_params).most_common()
    elif param in ['city', 'country']:
        pure_only = []
        for i in all_params:
            pure_only.append (i['title'])
        counter_result = Counter(pure_only).most_common()
    print (f'Всего в паблике: {counter_result}')
    return response, counter_result

## Команда помощи ши-тцу

In [ ]:
access_token = ''

In [ ]:
gender_counts (191438883, access_token)

In [ ]:
param_counts (191438883, access_token, 'bdate')

In [ ]:
param_counts (191438883, access_token, 'city')

In [ ]:
param_counts (191438883, access_token, 'country')

In [ ]:
param_counts (191438883, access_token, 'home_town')